# Pre-processing of signals from a given catalog

This file is intended to provide some tools for pre-processing a signal catalog. Load the signals, producing dataframes, apply filters, resampling, etc.

In [ ]:
def load_abylkairov(path,plot,wave_type,fs,Dinkpc):
  df = pd.read_csv(path)
  if wave_type == 0:
    GR_waves = df[df['GR_or_GREP'] == 0]

  elif wave_type == 1:
    GR_waves = df[df['GR_or_GREP'] == 1]

  else:
    print('Choose either 0 (GR waves) or 1 (GREP waves)')
  #Convert Distance to meters
  kpc2m    = 3.08567758128e+19
  D = Dinkpc * kpc2m

  ab_catalog = {}
  for s in range(452):
    signal = GR_waves[GR_waves['sample_id'] == s]
    h_ab = np.array(signal['amplitude'])/(100*D) #Convert to meters
    t_ab = np.array(signal['t(ms)'])/1000  #Convert to seconds

    t_ab, h_ab     = sn_resample_wave(t_ab, h_ab, fs) #Resample to fs sampling rate

    ab_catalog['h'+str(s)] = h_ab
    ab_catalog['t'+str(s)] = t_ab

  if plot:
    for s in range(452):
      t,h = ab_catalog['t' + str(s)],ab_catalog['h' + str(s)]
      plt.plot(t,h)
      plt.xlim(-0.005,0.007)
      plt.xlabel('Time (s)',fontsize=14)
      plt.ylabel('Strain',fontsize=14)
      plt.title(f'Abylkairov Signals @ {Dinkpc} kpc ')

  return ab_catalog

In [ ]:
# 'wrong' signals
a = [ 82,  83,  84,  88, 104, 105, 106, 178, 179, 181, 200, 201, 202, 207,
       223, 320, 323, 403, 405, 406, 407, 408, 426, 427, 428, 430, 431]

In [ ]:
def prepare_signal(catalog,tini,tend,s,fs, filter):
  t, h = catalog['t' + str(s)], catalog['h' + str(s)]
# Generate strain time-series
  hs = pycbc.types.TimeSeries(initial_array=h, delta_t=1.0/fs, epoch=t[0])

  # Generate a zeros time series to inject signal
  tzeros = np.arange(tini, tend, 1/fs)
  # Corrected: Use only the required arguments for CB when generating the zero time series
  # Assuming a default set of parameters for generating the zero time series for injection
  hzeros = pycbc.types.TimeSeries(initial_array=tzeros*0, delta_t=1.0 / fs, epoch=tzeros[0])

  # Get a time-series from h and inject
  hs = hzeros.inject(hs, copy=True)

  #Apply low-pass filter to signal
  if filter:
    fir_signal = lowpass_fir(hs, 300, 13)
    return fir_signal
  return hs


In [ ]:
def generate_x(hs,duration,tini,tend,fs):
  gps = 1256677376 + random.randint(0, 4000)
  segment = (int(gps), int(gps) + tend-tini)

  ldata = TimeSeries.fetch_open_data('L1', *segment, sample_rate=fs, verbose=False)
  nt = pycbc.types.TimeSeries(initial_array=ldata.value, delta_t=1.0 / fs)
  nt.start_time = tini

  x = hs + nt
  return x


In [ ]:
def whitening(x):
  welch = psd_est.welch
  if callable(psd_est.interpolate):
      interpolate = psd_est.interpolate
  else:
  # Si es módulo, toma la función interna
      interpolate = psd_est.interpolate.interpolate

  psd = interpolate(welch(x), 1.0 / x.duration)
  white_strain = ((x.to_frequencyseries() / psd ** 0.5).to_timeseries())*10e-24
  white_strain_cropped = white_strain.crop(1, 1)
  smooth = highpass_fir(white_strain_cropped, 35, 8)
  smooth = lowpass_fir(smooth, 300, 7)

  return smooth
